<a href="https://colab.research.google.com/github/bbutka/CEC595/blob/main/L05a_Pitfalls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will train a multi-layer perceptron and a logistic regression on our ship data

We load our libraries below

In [1]:
import csv
import random
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

Below we download our ship data

In [2]:
!wget https://raw.githubusercontent.com/bbutka/CEC595/main/data/ship.csv

--2021-01-09 19:13:19--  https://raw.githubusercontent.com/bbutka/CEC595/main/data/ship.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘ship.csv’

ship.csv            100%[===================>]  59.76K  --.-KB/s    in 0.01s   

2021-01-09 19:13:19 (4.32 MB/s) - ‘ship.csv’ saved [61194/61194]



Next, we write a script to process the data 

In [ ]:
first = True
with open("ship.csv") as f:
    csvdata = csv.reader(f, delimiter=',')
    data = []
    for row in csvdata:
      if not first: data += [row]
      first = False

array = []
for col in range(len(data[0])):
  array += [{}]
  new = 0
  for i in range(len(data)):	
     line = data[i]		
     if line[col] not in array[col]:
      array[col][line[col]] = new
      new += 1  
alldat = []
alllabs = []
for line in data:
  alllabs += [int(line[1])]
  if line[5] == '': line[5] = '50'
  alldat += [ [int(line[2]), array[4][line[4]], float(line[5]), int(line[6]), int(line[7]), float(line[9]), line[11]=='S', line[11]=='C', line[11]=='Q' ]]
feats = ['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked S', 'Embarked C', 'Embarked Q']

With our data processed, we create a `trainmask` to randomly separate our training from test data.  We will use the mask to get the training and test data.

In [ ]:
trainmask = [random.randint(0,2) for i in range(len(alldat))]

traindat = [alldat[i] for i in range(len(alldat)) if trainmask[i]<2]
trainlabs = [alllabs[i] for i in range(len(alldat)) if trainmask[i]<2]
testdat = [alldat[i] for i in range(len(alldat)) if trainmask[i]==2]
testlabs = [alllabs[i] for i in range(len(alldat)) if trainmask[i]==2]

We next will train a multi-layer perceptron with 60 hidden units to classify the data and print the accuracy.

In [ ]:
nhidden = 60
clf = MLPClassifier(hidden_layer_sizes=[nhidden], max_iter = 50000)
clf = clf.fit(traindat, trainlabs)
pred = clf.predict(testdat)
[sum([pred[i] != testlabs[i] for i in range(len(testlabs))]) / len(testlabs)]


[0.19047619047619047]

Next we will calculate how much higher the predictions are for females vs. males.

In [ ]:
# feats = ['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked S', 'Embarked C', 'Embarked Q']

imp = []
for v in alldat:
  real = v[1]
  v[1] = 0
  asmale = clf.predict_proba([v])[0][1]
  v[1] = 1
  asfemale = clf.predict_proba([v])[0][1]
  v[1] = real
  imp += [ asfemale-asmale ]
  
print(sum(imp)/len(imp))

0.4230867162154019


Next we will train a logistic regression and print the accuracy of the model on the train and test datasets

In [ ]:
clf = LogisticRegression(max_iter = 500)

clf.fit(traindat, trainlabs)  

pred = clf.predict(traindat)
trainerr = sum([pred[i] != trainlabs[i] for i in range(len(trainlabs))]) / len(trainlabs)
pred = clf.predict(testdat)
testerr = sum([pred[i] != testlabs[i] for i in range(len(testlabs))]) / len(testlabs)

print(trainerr, testerr)

0.2152777777777778 0.17777777777777778


Finally we'll plot the coefficients of the logistic regression model

In [ ]:
for i in range(len(feats)):
  print(feats[i], clf.coef_[0][i])


Pclass -0.8591194432385889
Sex 2.5902801936563105
Age -0.02604913505612622
SibSp -0.34447690011474924
Parch -0.20977293343256473
Fare 0.0022917046260743585
Embarked S -0.3617682332239227
Embarked C 0.04286405796040166
Embarked Q 0.18103820238828897
